In [1]:
import polars as pl
from core_pro.ultilities import make_sync_folder, update_df
from core_eda import TextEDA
from rich import print
from tqdm.auto import tqdm
from func import find_keyword_pipeline
from pattern import pattern

sh = '1P29Ql-SqjokycVjwXlj_2H9L1g0wOZApPfff3bl8muM'
path = make_sync_folder('nlp/ner')

In [2]:
df = (
    pl.read_parquet(path / 'tag_ner.parquet')
    .with_columns(pl.col('brand_name').fill_null('No brand'))
)
print(f'Data shape: {df.shape}, Total items: {df['item_id'].n_unique():,.0f}')
df.head()

Data shape: (10000000, 10), Total items: 4,467,822

item_id,item_name,level1_global_be_category,level2_global_be_category,level3_global_be_category,level4_global_be_category,level5_global_be_category,brand_name,attribute_name,attribute_value
i64,str,str,str,str,str,str,str,str,str
149946,"""Áo sơ mi cổ trụ""","""Women Clothes""","""Tops""","""Shirts & Blouses""",null,null,"""No brand""","""Neckline""","""Cổ sơ mi"""
606151,"""Quần legging Nhật""","""Women Clothes""","""Pants & Leggings""","""Leggings & Treggings""",null,null,"""No brand""","""Pattern""","""Trơn"""
675528,"""Chân váy bút chì cho cô nàng c…","""Women Clothes""","""Skirts""",null,null,null,"""No brand""","""Dress/Skirt Style""","""Váy bút chì"""
675528,"""Chân váy bút chì cho cô nàng c…","""Women Clothes""","""Skirts""",null,null,null,"""No brand""","""Occasion""","""Công việc"""
675528,"""Chân váy bút chì cho cô nàng c…","""Women Clothes""","""Skirts""",null,null,null,"""No brand""","""Pattern""","""Trơn"""


In [3]:
item = df['item_name'].str.to_lowercase()
keyword = df['attribute_value'].fill_null('None').str.to_lowercase()
lst_position = find_keyword_pipeline(item, keyword)

item = df['item_name'].str.to_lowercase()
keyword = df['brand_name'].str.to_lowercase()
lst_brand= find_keyword_pipeline(item, keyword)

  0%|          | 0/10000000 [00:00<?, ?it/s]

  0%|          | 0/10000000 [00:00<?, ?it/s]

In [4]:
df = df.with_columns(
    pl.Series('position_value', lst_position),
    pl.Series('position_brand', lst_brand),
)
df.head()

item_id,item_name,level1_global_be_category,level2_global_be_category,level3_global_be_category,level4_global_be_category,level5_global_be_category,brand_name,attribute_name,attribute_value,position_value,position_brand
i64,str,str,str,str,str,str,str,str,str,list[list[i64]],list[list[i64]]
149946,"""Áo sơ mi cổ trụ""","""Women Clothes""","""Tops""","""Shirts & Blouses""",null,null,"""No brand""","""Neckline""","""Cổ sơ mi""",null,null
606151,"""Quần legging Nhật""","""Women Clothes""","""Pants & Leggings""","""Leggings & Treggings""",null,null,"""No brand""","""Pattern""","""Trơn""",null,null
675528,"""Chân váy bút chì cho cô nàng c…","""Women Clothes""","""Skirts""",null,null,null,"""No brand""","""Dress/Skirt Style""","""Váy bút chì""","[[5, 16]]",null
675528,"""Chân váy bút chì cho cô nàng c…","""Women Clothes""","""Skirts""",null,null,null,"""No brand""","""Occasion""","""Công việc""",null,null
675528,"""Chân váy bút chì cho cô nàng c…","""Women Clothes""","""Skirts""",null,null,null,"""No brand""","""Pattern""","""Trơn""",null,null


In [5]:
df_ner = df.filter(~pl.col('position_value').is_null())
df_ner

item_id,item_name,level1_global_be_category,level2_global_be_category,level3_global_be_category,level4_global_be_category,level5_global_be_category,brand_name,attribute_name,attribute_value,position_value,position_brand
i64,str,str,str,str,str,str,str,str,str,list[list[i64]],list[list[i64]]
675528,"""Chân váy bút chì cho cô nàng c…","""Women Clothes""","""Skirts""",null,null,null,"""No brand""","""Dress/Skirt Style""","""Váy bút chì""","[[5, 16]]",null
1026939,"""Bộ 3 hộp đựng đồ cá nhân cỡ lớ…","""Home & Living""","""Home Organizers""","""Storage Boxes, Bags & Baskets""",null,null,"""No brand""","""Storage Type""","""Hộp""","[[5, 8]]",null
2085510,"""Áo kiểu cotton Hàn Quốc""","""Women Clothes""","""Tops""","""Shirts & Blouses""",null,null,"""No brand""","""Material""","""Cotton""","[[8, 14]]",null
3258375,"""Đầm xòe in hoa đào""","""Women Clothes""","""Dresses""",null,null,null,"""No brand""","""Pattern""","""Hoa""","[[11, 14]]",null
4698018,"""Combo Bàn phím và chuột không …","""Computers & Accessories""","""Keyboards & Mice""","""Keyboards""",null,null,"""Motospeed""","""Connection Type""","""Không dây""","[[24, 33]]","[[34, 43]]"
…,…,…,…,…,…,…,…,…,…,…,…
11899962957,"""Áo Thun Tay Lỡ Unisex Nam Nữ, …","""Women Clothes""","""Tops""","""T-shirts""",null,null,"""No brand""","""Pattern""","""In""","[[59, 61]]",null
11902427813,"""Set 28/50/100 Dây Buộc Tóc His…","""Fashion Accessories""","""Hair Accessories""","""Hair Ties, Ribbons & Scrunchie…",null,null,"""No brand""","""Gender""","""Nữ""","[[67, 69]]",null
11903147014,"""✷™✣Giày nữ cỡ lớn 41-43 Các mẫ…","""Sports & Outdoors""","""Sports Footwear""","""Running Shoes""",null,null,"""No brand""","""Gender""","""Nữ""","[[8, 10]]",null


In [6]:
import re

lst = ['SKU' if re.findall(r'\d+', text) else None for text in df_ner['attribute_value']]
lst = ['SKU' if l else n for l, n in zip(lst, df_ner['attribute_name'])]
for p in tqdm(pattern):
    k, v = list(p.items())[0]
    for token in k.split():
        set_token = {token}
        lst = [v if set_token.intersection(text.split()) else text for text in lst]
df_ner = df_ner.with_columns(pl.Series('attribute_name_patterns', lst))

  0%|          | 0/55 [00:00<?, ?it/s]

In [7]:
# check if ner tag is duplicated
tmp = (
    df_ner.group_by(['attribute_name', 'attribute_name_patterns']).agg(pl.col('attribute_value').unique())
    .with_columns(
        pl.col('attribute_value').list.join(', ').alias('all_name'),
        pl.col('attribute_value').list.len().alias('len')
    )
    .drop(['attribute_value'])
    .sort('len', descending=True)
)
update_df(tmp, 'ner_value', sh, start='A1')
tmp

🐶 Sheet There are 3 sheets

['ner_l1', 'ner_l1_value', 'ner_value']

🐶 Sheet Update values at: ner_value!A1:D

attribute_name,attribute_name_patterns,all_name,len
str,str,str,u32
"""Material""","""Material""","""Lụa, Carton, Sợi dệt, Len, Dệt…",83
"""Flavour""","""Flavour""","""Thịt bò, Cola, Trái cây, Phô m…",82
"""Nutrient Type""","""Type""","""Selenium, Nghệ, vitamin E, vit…",26
"""Region of Origin""","""Region""","""Hàn Quốc, Malaysia, Úc, Đài Lo…",25
"""Lighting Type""","""Type""","""Đèn năng lượng mặt trời, Đèn n…",17
…,…,…,…
"""Antenna Type""","""Type""","""Trong nhà""",1
"""TV Mount Feature""","""Feature""","""Giá treo tường""",1
"""Pan Type""","""Type""","""Chảo xào""",1


In [8]:
# check if ner is duplicated
tmp = (
    df_ner.group_by(['attribute_value']).agg([
        pl.col('attribute_name_patterns').unique(),
        pl.col('item_id').n_unique()
    ])
    .with_columns(
        pl.col('attribute_name_patterns').list.join(', ').alias('all_name'),
        pl.col('attribute_name_patterns').list.len().alias('len'),
    )
    .drop(['attribute_name_patterns'])
    .filter(pl.col('len') > 1)
    .sort('len', descending=True)
)
update_df(tmp, 'ner_value', sh, start='G1')
tmp

🐶 Sheet There are 3 sheets

['ner_l1', 'ner_l1_value', 'ner_value']

🐶 Sheet Update values at: ner_value!G1:J

attribute_value,item_id,all_name,len
str,u32,str,u32
"""Cam""",127,"""Flavour, Type, Colour""",3
"""Đen""",37,"""Style, Colour, Material""",3
"""USB""",231,"""Feature, Type, SKU""",3
"""Sáp""",38,"""Type, SKU, Material""",3
"""Khóa Zip""",10,"""Bag Closure, Type, Wallet Clos…",3
…,…,…,…
"""Chân""",40,"""Table Base, Application Area""",2
"""Bạc""",30504,"""Material, Colour""",2
"""C""",62,"""Size, Type""",2


In [9]:
# check attribute_name words
tmp = (
    df_ner
    .group_by(['level1_global_be_category', 'attribute_name'])
    .agg([
        pl.col('attribute_value').unique(),
        pl.col('item_id').count()
    ])
    .with_columns(pl.col('attribute_value').list.join(', '))
    # .with_columns((pl.col('count') / pl.col('item_id')).alias('pct'))
)
update_df(tmp, 'ner_l1_value', sh, start='A1')
tmp

🐶 Sheet There are 3 sheets

['ner_l1', 'ner_l1_value', 'ner_value']

🐶 Sheet Update values at: ner_l1_value!A1:D

level1_global_be_category,attribute_name,attribute_value,item_id
str,str,str,u32
"""Home & Living""","""Table Type""","""Bàn văn phòng, Bàn trang điểm,…",1994
"""Mobile & Gadgets""","""Distance""","""3KM, 5KM, 8KM""",9
"""Home & Living""","""Formulation""","""Dạng Kem, Dạng xịt, Lỏng, Bột,…",258
"""Men Clothes""","""Socks Type""","""Vớ thể thao""",125
"""Baby & Kids Fashion""","""Region of Origin""","""Nhật Bản, Hàn Quốc, Mỹ, Đài Lo…",485
…,…,…,…
"""Home Appliances""","""Fan Speed""","""Tốc độ 3""",1
"""Men Clothes""","""Top Fit Type""","""Rộng, Cổ điển, Cơ bắp, Oversiz…",1732
"""Food & Beverages""","""Noodle Type""","""Ramen, Bún tàu, Mì ăn liền, Bú…",109


In [10]:
# check attribute_name scale
tmp = (
    df_ner
    .group_by('level1_global_be_category')
    .agg([
        pl.col('attribute_name').value_counts(),
        pl.col('item_id').count()
    ])
    .explode(['attribute_name'])
    .unnest('attribute_name')
    .with_columns((pl.col('count') / pl.col('item_id')).alias('pct'))
    .filter(pl.col('pct') > 0.05)
)
update_df(tmp, 'ner_l1', sh, start='A1')
tmp

🐶 Sheet There are 3 sheets

['ner_l1', 'ner_l1_value', 'ner_value']

🐶 Sheet Update values at: ner_l1!A1:E

level1_global_be_category,attribute_name,count,item_id,pct
str,str,u32,u32,f64
"""Books & Magazines""","""Language""",1010,1546,0.653299
"""Books & Magazines""","""Cover Type""",517,1546,0.334411
"""Tickets, Vouchers & Services""","""Region""",3,6,0.5
"""Tickets, Vouchers & Services""","""Number of Nights""",1,6,0.166667
"""Tickets, Vouchers & Services""","""Voucher Form""",1,6,0.166667
…,…,…,…,…
"""Fashion Accessories""","""Material""",55486,155464,0.356906
"""Fashion Accessories""","""Hat Style""",11527,155464,0.074146
"""Women Clothes""","""Material""",298740,599039,0.498699
